In [0]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.linear_model as lm
from sklearn import preprocessing
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve,auc,recall_score,precision_score,accuracy_score,f1_score
from sklearn.metrics import confusion_matrix,average_precision_score,recall_score
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
df=pd.read_csv("titanic1.csv")
df.shape
df.info()

(1310, 14)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 14 columns):
pclass       1309 non-null float64
survived     1309 non-null float64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null float64
parch        1309 non-null float64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(7), object(7)
memory usage: 143.4+ KB


In [0]:
df.head()
import statistics
titanic=df
#titanic=df.fillna({'Age':df['age'].median(),'embarked':statistics.mode})

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [0]:
titanic.drop(['name','boat','body','home.dest','ticket','cabin'],axis=1,inplace=True)

In [0]:
titanic.head()
titanic.rename(columns={'sex':'gender','sibsp':'siblings','parch':'parents_child'}, inplace=True)

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1.0,1.0,female,29.0000,0.0,0.0,211.3375,S
1,1.0,1.0,male,0.9167,1.0,2.0,151.5500,S
2,1.0,0.0,female,2.0000,1.0,2.0,151.5500,S
3,1.0,0.0,male,30.0000,1.0,2.0,151.5500,S
4,1.0,0.0,female,25.0000,1.0,2.0,151.5500,S


In [0]:
titanic.head()

,pclass,survived,gender,age,siblings,parents_child,fare,embarked
0,1.0,1.0,female,29.0000,0.0,0.0,211.3375,S
1,1.0,1.0,male,0.9167,1.0,2.0,151.5500,S
2,1.0,0.0,female,2.0000,1.0,2.0,151.5500,S
3,1.0,0.0,male,30.0000,1.0,2.0,151.5500,S
4,1.0,0.0,female,25.0000,1.0,2.0,151.5500,S


In [0]:
titanic[titanic['embarked'].isnull()]

,pclass,survived,gender,age,siblings,parents_child,fare,embarked
168,1.0,1.0,female,38.0,0.0,0.0,80.0,NaN
284,1.0,1.0,female,62.0,0.0,0.0,80.0,NaN
1309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
titanic['embarked']=titanic['embarked'].fillna(titanic['embarked'].mode()[0])

In [0]:
titanic[titanic['embarked'].isnull()]
#titanic['embarked'].isnull()

,pclass,survived,gender,age,siblings,parents_child,fare,embarked


In [0]:

titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 8 columns):
pclass           1309 non-null float64
survived         1309 non-null float64
gender           1309 non-null object
age              1046 non-null float64
siblings         1309 non-null float64
parents_child    1309 non-null float64
fare             1308 non-null float64
embarked         1310 non-null object
dtypes: float64(6), object(2)
memory usage: 82.0+ KB


In [0]:
titanic_pivot=pd.pivot_table(titanic,values='age',index='gender',columns='pclass',aggfunc=np.mean)
titanic_pivot
by_gender_class=titanic.groupby(['gender','pclass'])
type(by_gender_class['age'])

pclass,1.0,2.0,3.0
gender,,,
female,37.037594,27.499191,22.185307
male,41.029250,30.815401,25.962273


pandas.core.groupby.generic.SeriesGroupBy

In [0]:
def impute_median(x):
  return x.fillna(x.median())

In [0]:
titanic.age=by_gender_class['age'].apply(impute_median)

titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 8 columns):
pclass           1309 non-null float64
survived         1309 non-null float64
gender           1309 non-null object
age              1309 non-null float64
siblings         1309 non-null float64
parents_child    1309 non-null float64
fare             1308 non-null float64
embarked         1310 non-null object
dtypes: float64(6), object(2)
memory usage: 82.0+ KB


In [0]:
titanic=titanic.fillna(titanic.median())
for x in titanic.columns:  
  if titanic[x].dtype=='object':
      titanic[x]=pd.Categorical(titanic[x]).codes
titanic.head()

,pclass,survived,gender,age,siblings,parents_child,fare,embarked
0,1.0,1.0,0,29.0000,0.0,0.0,211.3375,2
1,1.0,1.0,1,0.9167,1.0,2.0,151.5500,2
2,1.0,0.0,0,2.0000,1.0,2.0,151.5500,2
3,1.0,0.0,1,30.0000,1.0,2.0,151.5500,2
4,1.0,0.0,0,25.0000,1.0,2.0,151.5500,2


In [0]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 8 columns):
pclass           1310 non-null float64
survived         1310 non-null float64
gender           1310 non-null int8
age              1310 non-null float64
siblings         1310 non-null float64
parents_child    1310 non-null float64
fare             1310 non-null float64
embarked         1310 non-null int8
dtypes: float64(6), int8(2)
memory usage: 64.1 KB


In [0]:
titanic=titanic[:-1]
titanic.info()
x=titanic.drop(['survived'],axis=1)
y=titanic['survived']
y.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
pclass           1309 non-null float64
survived         1309 non-null float64
gender           1309 non-null int8
age              1309 non-null float64
siblings         1309 non-null float64
parents_child    1309 non-null float64
fare             1309 non-null float64
embarked         1309 non-null int8
dtypes: float64(6), int8(2)
memory usage: 64.0 KB


0    1.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: survived, dtype: float64

In [0]:
x.head()

,pclass,gender,age,siblings,parents_child,fare,embarked
0,1.0,0,29.0000,0.0,0.0,211.3375,2
1,1.0,1,0.9167,1.0,2.0,151.5500,2
2,1.0,0,2.0000,1.0,2.0,151.5500,2
3,1.0,1,30.0000,1.0,2.0,151.5500,2
4,1.0,0,25.0000,1.0,2.0,151.5500,2


In [0]:
#we will also split our data into train and test datasets and Build the model on train dataset and apply the model on test datset.
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import pydotplus
from sklearn.model_selection import GridSearchCV
my_max_depth=[2,5,10,15,20,25]
my_criterion=['gini','entropy']
#BUILDING THE MODEL
mytreeone=DecisionTreeClassifier()
#CROSS VALIDATION TELLS HOW A MODEL PERFORMS ON A DATASET USING MULTIPLE SAMPLES OF TRAIN DATA
grid=GridSearchCV(estimator=mytreeone,cv=3,param_grid=dict(max_depth=my_max_depth,criterion=my_criterion))
grid.fit(x,y)

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                   

In [0]:
#best accuracy score
grid.best_params_

{'criterion': 'gini', 'max_depth': 2}

In [0]:
#best parameters for the model
results=pd.DataFrame(grid.cv_results_)

In [0]:
results.to_csv("results.csv")